In [5]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from IPython.core.display import clear_output
from urllib.parse import quote
import chromedriver_autoinstaller

In [6]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=opt)

In [7]:
errors = []

In [8]:
def expand_shadow_element(element):
    shadow_root = driver.execute_script('return arguments[0].shadowRoot',
                                        element)
    return shadow_root

In [9]:
def get_rt_movie_url(movie, year):
    url = r"https://www.rottentomatoes.com/search?search=" + quote(movie)

    driver.get(url)
    try:
        root1 = driver.find_element_by_tag_name('search-result-container')
        shadow_root1 = expand_shadow_element(root1)

        root2 = shadow_root1.find_element_by_css_selector(
            "search-result[type='movie']")
        shadow_root2 = expand_shadow_element(root2)

        html_of_interest = driver.execute_script(
            'return arguments[0].innerHTML', shadow_root2)
        sel_soup = BeautifulSoup(html_of_interest, 'html.parser')

        movie_url = sel_soup.find('media-row', {'releaseyear': year})
    except NoSuchElementException:
        errors.append(movie)
        print("Movie not found!!")
        return None

    #driver.quit()

    #print(type(movie_url))
    if movie_url is not None:
        print("get_rt_movie_url\nMovie:{},url:{}".format(
            movie, movie_url['url']))
        sleep(randint(1, 3))
        return movie_url['url']
    else:
        print("Movie Url not found!")
        sleep(randint(1, 3))
        return None

In [10]:
def get_rt_scores(movie, year):
    movie_url = get_rt_movie_url(movie, year)
    a = np.nan
    c = np.nan

    if movie_url is not None:
        try:
            response = get(movie_url)
            html_soup = BeautifulSoup(response.text, 'html.parser')

            x = html_soup.find('score-board')

            sleep(randint(1, 3))

            a = x['audiencescore']
            c = x['tomatometerscore']
        except TypeError:
            errors.append(movie)
            print("TypeError raised")

        print("get_rt_scores\nMovie:{},critic:{},audience:{}".format(
            movie, c, a))

    return [c, a]

In [11]:
imdb_df = pd.read_csv('imdb_meta.csv')

In [12]:
imdb_df

,movie,year,runtime,genre,imdb,meta,imdb_votes
0,The Shawshank Redemption,1994,142 min,Drama,9.3,80,2360495
1,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,84,2319761
2,Inception,2010,148 min,"Action, Adventure, Sci-Fi",8.8,74,2083710
3,Fight Club,1999,139 min,Drama,8.8,66,1868351
4,Pulp Fiction,1994,154 min,"Crime, Drama",8.9,94,1839615
...,...,...,...,...,...,...,...
8887,Falling Overnight,2011,86 min,"Drama, Romance",6.7,40,1002
8888,Fremde Haut,2005,97 min,Drama,7.0,60,1002
8889,Bam gua nat,2008,144 min,Drama,7.1,73,1001
8890,Awful Nice,2013,92 min,Comedy,4.9,42,1000


In [13]:
r=5

In [14]:
rt_df = imdb_df.copy()
test_df = rt_df.sample(r)

In [15]:
movies = imdb_df['movie']

In [16]:
years = imdb_df['year']

In [17]:
tomato = {}

In [21]:
imdb_df.iloc[6641]

movie                     Band Aid
year                          2017
runtime                     91 min
genre         Comedy, Drama, Music
imdb                           6.6
meta                            67
imdb_votes                    5214
Name: 6641, dtype: object

In [22]:
flag = True
k=6642

In [ ]:
while flag is True:
    try:
        for i in range(k,8893):
            clear_output(wait=True)
            print("i:{}, movie:{}".format(i, movies[i]))
            a = get_rt_scores(movies[i], years[i])
            tomato[movies[i]] = a
            tom_df = make_db(tomato)
            if i == 8892:
                flag = False
    except:
        k=i
        driver = webdriver.Chrome()

i:4902, movie:Dangerous Lies
get_rt_movie_url
Movie:Dangerous Lies,url:https://rottentomatoes.com/m/771521317


In [ ]:
for i in range(6642,8000):
            clear_output(wait=True)
            print("i:{}, movie:{}".format(i, movies[i]))
            a = get_rt_scores(movies[i], years[i])
            tomato[movies[i]] = a
            tom_df = make_db(tomato)

i:6747, movie:Mental


In [19]:
i

5671

In [20]:
tomato

{'Contracted': ['53', '29'],
 'Hell Fest': ['40', '41'],
 'Leatherface: Texas Chainsaw Massacre III': ['19', '31'],
 'Better Living Through Chemistry': ['22', '38'],
 'Breathe In': ['56', '41'],
 'Barbershop 2: Back in Business': ['68', '48'],
 'Welcome to Collinwood': ['55', '56'],
 'The Road Within': ['45', '69'],
 'Battle in Seattle': ['56', '65'],
 'Body of Evidence': ['8', '23'],
 'Medianeras': ['67', '83'],
 'Come to Daddy': ['86', '75'],
 'My Fellow Americans': ['47', '64'],
 'Agent Cody Banks 2: Destination London': ['14', '31'],
 'Mary Reilly': ['26', '40'],
 'The Lifeguard': ['16', '26'],
 'Senseless': ['6', '45'],
 'Cats & Dogs: The Revenge of Kitty Galore': ['13', '38'],
 'A Fantastic Fear of Everything': ['34', '35'],
 'A Futile and Stupid Gesture': ['67', '71'],
 'Entebbe': ['', ''],
 'Keepers': ['', ''],
 'Unforgettable': ['28', '33'],
 'Ararat': ['55', '71'],
 'The Musketeer': ['11', '33'],
 'The Wild Thornberrys Movie': ['80', '51'],
 'For Your Consideration': ['52', '

In [24]:
def make_db(d):
    df = pd.DataFrame.from_dict(d, orient='index', columns=['rt_critic', 'rt_audience'])
    df.reset_index(inplace=True)
    df.rename(columns={'index':'movie'}, inplace=True)
    df.to_csv('rt5.csv', index=False)
    return df

In [64]:
tom_df = make_db(tomato)

In [29]:
tom_df = pd.read_csv('rt.csv', header=None)

In [30]:
tom_df.set_index(0, inplace=True)

In [31]:
tom_df.index.name = None

In [32]:
tom_df

,1,2
movie,rt_critic,rt_audience
The Shawshank Redemption,91,98
The Dark Knight,94,94
Inception,87,91
Fight Club,79,96
...,...,...
Safe Haven,14,67
The Queen,NaN,NaN
The Commuter,55,40
The Manchurian Candidate,80,63


In [33]:
tomato = tom_df.to_dict(orient='index')

In [34]:
tomato

{'movie': {1: 'rt_critic', 2: 'rt_audience'},
 'The Shawshank Redemption': {1: '91', 2: '98'},
 'The Dark Knight': {1: '94', 2: '94'},
 'Inception': {1: '87', 2: '91'},
 'Fight Club': {1: '79', 2: '96'},
 'Pulp Fiction': {1: '92', 2: '96'},
 'Forrest Gump': {1: '71', 2: '95'},
 'The Matrix': {1: '88', 2: '85'},
 'The Lord of the Rings: The Fellowship of the Ring': {1: '91', 2: '95'},
 'The Lord of the Rings: The Return of the King': {1: '93', 2: '86'},
 'Interstellar': {1: '72', 2: '86'},
 'The Dark Knight Rises': {1: '87', 2: '90'},
 'The Lord of the Rings: The Two Towers': {1: '95', 2: '95'},
 'Seven': {1: '83', 2: '95'},
 'Django Unchained': {1: '87', 2: '91'},
 'Gladiator': {1: '77', 2: '87'},
 'Batman Begins': {1: '84', 2: '94'},
 'The Silence of the Lambs': {1: '96', 2: '95'},
 'Inglourious Basterds': {1: '89', 2: '88'},
 'The Avengers': {1: '91', 2: '91'},
 'Saving Private Ryan': {1: '93', 2: '95'},
 "Schindler's List": {1: '97', 2: '97'},
 'The Wolf of Wall Street': {1: '79', 2

In [8]:
t=3915

In [9]:
m,y = movies[t], str(years[t])

In [10]:
movies[t]

'Kraftidioten'

In [ ]:
print("{},{}".format(m,y))

In [ ]:
url = get_rt_movie_url(m,y)

In [ ]:
get_rt_scores(m, y)